# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [14]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier


In [4]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [6]:
spaceship_clean=spaceship.dropna()
print(spaceship_clean.isnull().sum())
spaceship_clean['Cabin'] = spaceship_clean['Cabin'].str[0]
spaceship_clean['Cabin']
spaceship_clean.drop(columns=['PassengerId', 'Name'], inplace=True)

non_numerical_cols = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']
# Apply one-hot encoding

spaceship_clean = pd.get_dummies(spaceship_clean, columns=non_numerical_cols, drop_first=True)

spaceship_clean = spaceship_clean.astype(int)


PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64


C:\Users\sabri\AppData\Local\Temp\ipykernel_28848\809873756.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spaceship_clean['Cabin'] = spaceship_clean['Cabin'].str[0]
C:\Users\sabri\AppData\Local\Temp\ipykernel_28848\809873756.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spaceship_clean.drop(columns=['PassengerId', 'Name'], inplace=True)


**Perform Train Test Split**

In [7]:
X, y = spaceship_clean.drop(columns=['Transported']), spaceship_clean['Transported']



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42
                                                    )
# we use the random state to make sure we select exactly the same rows

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [23]:


# Base model
base_model = DecisionTreeClassifier(random_state=42)

# Bagging (with replacement)
bagging = BaggingClassifier(
    base_model,
    n_estimators=50,
    max_samples=0.8,
    bootstrap=True,         # ← Bagging
    random_state=42
)

# Pasting (without replacement)
pasting = BaggingClassifier(
    base_model,
    n_estimators=50,
    max_samples=0.8,
    bootstrap=False,        # ← Pasting
    random_state=42
)

# Fit both models
bagging.fit(X_train, y_train)
pasting.fit(X_train, y_train)

# Evaluate
y_pred_bag = bagging.predict(X_test)
y_pred_paste = pasting.predict(X_test)

print(f"Bagging accuracy: {accuracy_score(y_test, y_pred_bag):.4f}")
print(f"Pasting accuracy: {accuracy_score(y_test, y_pred_paste):.4f}")


# Predict and evaluate

print(f"Bagging : \n {classification_report(y_test, y_pred_bag)}")

print(f"pasting: \n {classification_report(y_test, y_pred_paste)}")

Bagging accuracy: 0.8033
Pasting accuracy: 0.7995
Bagging : 
               precision    recall  f1-score   support

           0       0.79      0.81      0.80       653
           1       0.81      0.80      0.80       669

    accuracy                           0.80      1322
   macro avg       0.80      0.80      0.80      1322
weighted avg       0.80      0.80      0.80      1322

pasting: 
               precision    recall  f1-score   support

           0       0.79      0.81      0.80       653
           1       0.81      0.79      0.80       669

    accuracy                           0.80      1322
   macro avg       0.80      0.80      0.80      1322
weighted avg       0.80      0.80      0.80      1322



- Random Forests

In [11]:

# Initialize the model
rf_model = RandomForestClassifier(
    n_estimators=100,       # number of trees
    max_depth=None,         # no depth limit
    random_state=42,        # for reproducibility
    n_jobs=-1               # use all CPU cores
)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Evaluate performance
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.8048
              precision    recall  f1-score   support

           0       0.79      0.82      0.81       653
           1       0.82      0.79      0.80       669

    accuracy                           0.80      1322
   macro avg       0.81      0.80      0.80      1322
weighted avg       0.81      0.80      0.80      1322



- Gradient Boosting

In [13]:
# Initialize the model
gb_model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)

# Train the model
gb_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = gb_model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.8071
              precision    recall  f1-score   support

           0       0.84      0.76      0.79       653
           1       0.78      0.86      0.82       669

    accuracy                           0.81      1322
   macro avg       0.81      0.81      0.81      1322
weighted avg       0.81      0.81      0.81      1322



- Adaptive Boosting

In [17]:

# Base learner
base_model = DecisionTreeClassifier(max_depth=1, random_state=42)

# AdaBoost classifier (corrected)
ada_model = AdaBoostClassifier(
    estimator=base_model,         # ← updated keyword
    n_estimators=100,
    learning_rate=0.5,
    random_state=42
)

# Train the model
ada_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = ada_model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.7700
              precision    recall  f1-score   support

           0       0.73      0.85      0.79       653
           1       0.83      0.69      0.75       669

    accuracy                           0.77      1322
   macro avg       0.78      0.77      0.77      1322
weighted avg       0.78      0.77      0.77      1322



Which model is the best and why?

In [25]:

# Model performance data
data = {
    'Model': ['Bagging', 'Pasting', 'Random Forest', 'Gradient Boosting', 'Adaptive Boosting'],
    'Accuracy': [0.8033, 0.7995, 0.8048, 0.8071, 0.7700],
    'Precision (0)': [0.79, 0.79, 0.79, 0.84, 0.73],
    'Recall (0)': [0.81, 0.81, 0.82, 0.76, 0.85],
    'F1-score (0)': [0.80, 0.80, 0.81, 0.79, 0.79],
    'Precision (1)': [0.81, 0.81, 0.82, 0.78, 0.83],
    'Recall (1)': [0.80, 0.79, 0.79, 0.86, 0.69],
    'F1-score (1)': [0.80, 0.80, 0.80, 0.82, 0.75]
}

# Create DataFrame
df = pd.DataFrame(data)

# Display the table
df

,Model,Accuracy,Precision (0),Recall (0),F1-score (0),Precision (1),Recall (1),F1-score (1)
0,Bagging,0.8033,0.79,0.81,0.80,0.81,0.80,0.80
1,Pasting,0.7995,0.79,0.81,0.80,0.81,0.79,0.80
2,Random Forest,0.8048,0.79,0.82,0.81,0.82,0.79,0.80
3,Gradient Boosting,0.8071,0.84,0.76,0.79,0.78,0.86,0.82
4,Adaptive Boosting,0.7700,0.73,0.85,0.79,0.83,0.69,0.75


 Best Overall: Gradient Boosting

- Highest accuracy (0.8071)

- Balanced macro F1-score (0.81)

- Strong performance on both classes:

- Class 0: high precision (0.84)

- Class 1: high recall (0.86)
